In [63]:
# @title Default title text
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dense
import tensorflow.keras.layers as tfl
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers

In [64]:

from tensorflow.keras.models import Model

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=True)

# Create a new model to extract features from the last fully connected layer
model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

print(model.summary())

def Extract_features(img_arr):
    feature = model.predict(img_arr, verbose=0)
    return feature


Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0  

In [65]:
import numpy as np
import cv2
import os
import random
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

# Define the number of parallel processes
NUM_PARALLEL_PROCESSES = 4

# Load pre-trained VGG16 model for feature extraction
base_model = VGG16(weights='imagenet', include_top=True)
feature_extractor = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

# Image Data Generator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    fill_mode='nearest'
)

DIRECTORY = '/content/drive/MyDrive/valid'
CATEGORY = ['cats', 'dogs']

data = []

# Function to process images and extract features
def process_images_and_features(img_paths, label):
    processed_images = []
    for img_path in img_paths:
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr, (224, 224))  # Resize the image to (224, 224)
        img_arr = np.expand_dims(img_arr, axis=0)  # Add batch dimension
        features = Extract_features(img_arr)
        processed_images.append((img_arr, features))
    return processed_images

# Process images and extract features
count = 0
for category in CATEGORY:
    folder = os.path.join(DIRECTORY, category).replace('\\', '/')
    label = CATEGORY.index(category)
    img_paths = [os.path.join(folder, img) for img in os.listdir(folder)]

    # Split image paths into batches for parallel processing
    batch_size = len(img_paths) // NUM_PARALLEL_PROCESSES
    img_path_batches = [img_paths[i:i+batch_size] for i in range(0, len(img_paths), batch_size)]

    # Process images and extract features in parallel
    for i,img_path in enumerate(img_paths):
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr, (224, 224))
        if len(img_arr.shape) == 3:  # Check if the image is color (has 3 channels)
            gray_image = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
            cv2.imwrite(img_path, gray_image)
        else:  # If the image is already grayscale, just save it without conversion
            cv2.imwrite(img_path, img_arr)

    processed_images = []
    for batch in img_path_batches:
        processed_images.extend(process_images_and_features(batch, label))
        count += len(batch)
        print("Processed {} images".format(count))

    # Augment the original images
    for img_arr, original_features in processed_images:
        data.append([img_arr, label, original_features])

# Shuffle the data
random.shuffle(data)

# Convert data array to numpy arrays X, Y, and features
X = np.array([item[0] for item in data])
Y = np.array([item[1] for item in data])
features = np.array([item[2] for item in data])



Processed 250 images
Processed 500 images
Processed 750 images
Processed 1000 images
Processed 1250 images
Processed 1500 images
Processed 1750 images
Processed 2000 images


In [66]:

features = np.squeeze(features, axis=1)
X_train,X_test,Y_train,Y_test=train_test_split(features,Y,test_size=0.2,random_state=42)
print(features.shape)
print(X_train.shape,"iiii",Y_train.shape)

(2000, 4096)
(1600, 4096) iiii (1600,)


In [78]:
model3 = tf.keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=(224, 224, 3)),
    tfl.Dense(units=128, activation="relu"),
    tfl.Dense(units=64, activation="relu"),
    tfl.Dense(units=1, activation="sigmoid")
])
optimizer3 = tf.keras.optimizers.Adam(learning_rate=0.001)

loss_function3 = tf.keras.losses.BinaryCrossentropy()

model3.compile(optimizer=optimizer3, loss=loss_function3, metrics=['accuracy'])
model3.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10
50/50 [==============================] - 2s 15ms/step - loss: 0.1797 - accuracy: 0.9369
Epoch 2/10
50/50 [==============================] - 1s 10ms/step - loss: 0.0483 - accuracy: 0.9800
Epoch 3/10
50/50 [==============================] - 1s 14ms/step - loss: 0.0210 - accuracy: 0.9937
Epoch 4/10
50/50 [==============================] - 1s 14ms/step - loss: 0.0094 - accuracy: 0.9975
Epoch 5/10
50/50 [==============================] - 1s 15ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 6/10
50/50 [==============================] - 1s 14ms/step - loss: 6.9478e-04 - accuracy: 1.0000
Epoch 7/10
50/50 [==============================] - 1s 14ms/step - loss: 3.0911e-04 - accuracy: 1.0000
Epoch 8/10
50/50 [==============================] - 1s 15ms/step - loss: 2.3001e-04 - accuracy: 1.0000
Epoch 9/10
50/50 [==============================] - 1s 15ms/step - loss: 1.8632e-04 - accuracy: 1.0000
Epoch 10/10
50/50 [==============================] - 1s 11ms/step - loss: 1.5363e-04 - accura

In [79]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, Y_train)

KNeighborsClassifier()

In [80]:
from sklearn.svm import SVC
svc_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svc_model.fit(X_train, Y_train)

SVC()

In [81]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, Y_train)

RandomForestClassifier(random_state=42)

In [82]:
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, Y_train)

DecisionTreeClassifier(random_state=42)

In [83]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_model.fit(X_train, Y_train)

GaussianNB()

In [84]:
from sklearn.metrics import accuracy_score
models_name=["KNN","Logistic regression","SVC","Random Forest","Decision Tree","Naive Bayes"]
models_accuracy=[]
models=[knn_model,model3,svc_model,rf_model,dt_model,nb_model]
for mod in models:
    predictions=mod.predict(X_test)
    Y_pred = [0 if p < 0.5 else 1 for p in predictions]
    models_accuracy.append(accuracy_score(Y_test,Y_pred))
chosen_model_index=np.argmax(models_accuracy)
chosen_model =models[chosen_model_index]
print(models_accuracy)
print(models_name[chosen_model_index])

13/13 [==============================] - 0s 4ms/step
[0.945, 0.9575, 0.955, 0.9575, 0.88, 0.94]
Logistic regression


In [85]:

def predict_model(X):
    # Ensure X is in the correct shape
    X = X.reshape(1, -1)  # Assuming X is flattened, reshape it to match the expected input shape
    pred = chosen_model.predict(X)
    predictions = [0 if p < 0.5 else 1 for p in pred]
    return predictions

predictions = predict_model(X_test[0])
print("Prediction:", predictions[0])
print("Actual Label:", Y_test[0])
print("Comparison:", predictions[0] == Y_test[0])
print("it's",CATEGORY[predictions[0]])
pred=chosen_model.predict(X_test)
predictions = [0 if p < 0.5 else 1 for p in pred]
print(np.sum(predictions==Y_test)/Y_test.shape[0])


1/1 [==============================] - 0s 23ms/step
Prediction: 1
Actual Label: 1
Comparison: True
it's dogs
13/13 [==============================] - 0s 3ms/step
0.9575
